# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### 1. Immigration Data


In [ ]:
# Immigration Data
immigration_fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(immigration_fname, 'sas7bdat', encoding="ISO-8859-1")
df_immigration.head()

In [ ]:
df_immigration.dtaddto.unique()

In [ ]:
df_immigration[["arrdate", "depdate", "dtadfile", "dtaddto", "entdepa", "entdepd", "entdepu", "matflag"]]

In [ ]:
df_immigration.isnull().sum()

|  Column Name | Description  |
|---|---|
|   **cicid**       |  Unique Identifier for the Immigration Data |
|   **i94yr**       |   Year |
|   **i94mon**      |   Month |
|   **i94cit**      |   City Code (Cirty of Origin)|
|   **i94res**      |   Country Code (Country of Residence)|
|   **i94port**     |   Port City of Entry into U.S.|
|   **arrdate**     |   Arrival Date in US (SAS date format)|
|   **i94mode**     |   Mode of transportation (1: Air, 2: Sea, 3: Land, 9: Not reported) |
|   **i94addr**     |   State of Arrival|
|   **depdate**     |   Departure Date from US |
|   **i94bir**      |   Age of Respondent in Years |
|   **i94visa**     |   Visa codes collapsed into three categories (1 = Business, 2 = Pleasure, 3 = Student)|
|   **count**       |   Used for Summary Statistics |
|   **dtadfile**    |   Character Date Field - Data of Arrival |
|   **visapost**    |   Department of State where where Visa was issued |
|   **occup**       |   Occupation that will be performed in U.S. |
|   **entdepa**     |   Arrival Flag - admitted or paroled into the U.S. |
|   **entdepd**     |   Departure Flag - Departed, lost I-94 or is deceased |
|   **entdepu**     |   Update Flag - Either apprehended, overstayed, adjusted to perm residence |
|   **matflag**     |   Match flag - Match of arrival and departure records |
|   **biryear**     |   4 digit year of birth |
|   **dtaddto**     |   Character Date Field - Date until which allowed to stay in U.S. |
|   **gender**      |   Non-immigrant sex |
|   **insnum**      |   INS number |
|   **airline**     |   Airline used to arrive in U.S. |
|   **admnum**      |   Admission Number |
|   **fltno**       |    Flight number of Airline used to arrive in U.S. |
|   **visatype**    |   Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |

### 2. Temperature Data

In [ ]:
temperature_fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(temperature_fname)
df_temperature.head()

In [ ]:
df_temperature.isnull().sum()

|  Column Name | Description  |
|---|---|
|   **dt**       |  Date in 'YYYY-MM-DD' format |
|   **AverageTemperature**       |   Average temperature in degrees |
|   **AverageTemperatureUncertainty**      |   Average temperature uncertainity in degrees |
|   **City**      |   City name |
|   **Country**      |   Country name |
|   **Latitude**      |   Latitude |
|   **Longitude**      |   Longitude |

### 3. U.S. Cities Deographic Data

In [ ]:
df_cities = pd.read_csv("us-cities-demographics.csv", delimiter=";")
df_cities.head()

In [ ]:
df_cities.isnull().sum()

|  Column Name | Description  |
|---|---|
|   **City**      |   City name |
|   **State**         |   U.S. State of city |
|   **Median Age**      |   Median of the age of population |
|   **Male Population** |   Size of male population |
|   **Female Population**      |   Size of female population |
|   **Total Population**      |   Size of total population |
|   **Number of Veterans**      |   Number of Veterans in city |
|   **Foreign-born**      |   Number of foreign-borns in city |
|   **Average Household Size**      |   Average size of house-hold |
|   **State Code**      |   Code of the state of city |
|   **Race**      |   Majority race in the city |
|   **Count**      |   Population of the majority race |

### 4. Airport Data

In [ ]:
df_airport = pd.read_csv("airport-codes_csv.csv")
df_airport.head()

In [ ]:
df_airport.info()

|  Column Name | Description  |
|---|---|
|   **ident**      |   Unique identifier |
|   **type**         |   Type of the airport |
|   **name**      |   Name of the airport |
|   **elevation_ft** |   Altitude of the airport |
|   **continent**      |   Continent of the airport |
|   **iso_country**      |   ISO Code of the country of the airport |
|   **iso_region**      |   ISO Code of the region of the airport |
|   **municipality**      |   City of the airport |
|   **gps_code**      |   GPS Code of the airport |
|   **iata_code**      |   IATA Code of the airport |
|   **local_code**      |   Local Code of the airport |
|   **coordinates**      |   GPS coordinates of the airport |

In [ ]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [ ]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("udacity-capstone-project-outputs/state_demographics")
df_spark.show(n=5)

In [ ]:
df_spark.printSchema

In [ ]:
# Performing cleaning tasks here
df_spark.createOrReplaceTempView("immigration_table")




In [ ]:
df_spark.head()

In [ ]:
immigration_table = spark.sql(""" 
        select distinct cicid as ID, arrdate as ARR_DATE, depdate as DEP_DATE, 
        i94port as ARR_CITYCODE, i94addr as ARR_STATECODE, i94mode as ARR_MODE, fltno as ARR_FLIGHT, airline as ARR_AIRLINE, 
        biryear as RES_BIRTHYEAR, i94res as RES_COUNTRYCODE, gender as RES_GENDER, 
        dtaddto as VISA_EXPIRYDATE, visapost as VISA_ISSUESTATE, i94visa as VISA_CATEGORY, visatype as VISA_TYPE
        from immigration_table    
        """)

In [ ]:
immigration_table.show(n=5)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In [1]:
from etl.etl import create_spark_session, etl_immigration, etl_temperature_by_country, etl_state_demographics

In [2]:
spark = create_spark_session()

In [ ]:
etl_immigration(spark)

In [3]:
etl_temperature_by_country(spark)

In [ ]:
etl_state_demographics(spark)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.